In [2]:
import os
import numpy as np
import pandas as pd
from __future__ import print_function
import matplotlib.pyplot as plt
import sys
from IPython.display import display, Image
from scipy import ndimage
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras import optimizers
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input
from sklearn import linear_model


Using TensorFlow backend.


In [157]:
##This notebook is built around using tensorflow as the backend for keras 
!pip install pillow
!KERAS_BACKEND=tensorflow python -c "from keras import backend"

Using TensorFlow backend.


In [158]:
##Dimensions of our images.# dimen 
# img_width, img_height = 150, 150

# train_data_dir = 'data/train'
# validation_data_dir = 'data/validation'

In [159]:
##Used to rescale the pixel values from [0, 255] to [0, 1] interval
datagen = ImageDataGenerator(rescale=1./255)

##Automagically retrieve images and their classes for train and validation sets
train_generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=16,
        class_mode='binary',
        classes = ['neutral', 'sadness', 'surprise', 'happiness', 'fear', 'anger', 'contempt', 'disgust'])

validation_generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=32,
        class_mode='binary',
        classes = ['neutral', 'sadness', 'surprise', 'happiness', 'fear', 'anger', 'contempt', 'disgust'])

Found 0 images belonging to 8 classes.
Found 0 images belonging to 8 classes.


In [160]:
with open("fer2013.csv") as f:
    content = f.readlines()
 
    lines = np.array(content)
 
    num_of_instances = lines.size
    print("number of instances: ",num_of_instances)


number of instances:  35888


In [3]:
filname='fer2013.csv'
label_map = ['Anger', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
def getData(filname):
    # images are 48x48 = 2304 size vectors
    # N = 35887
    Y = []
    X = []
    first = True
    for line in open(filname):
        if first:
            first = False
        else:
            row = line.split(',')
            Y.append(int(row[0]))
            X.append([int(p) for p in row[1].split()])

    X, Y = np.array(X) / 255.0, np.array(Y) # scaling is already done here 
    return X, Y 

X, Y = getData(filname)
print(X.shape)
print(Y.shape)
print(len(set(Y)))
num_class=len(set(Y))

#check if classes are balanced
def balance_class(Y): 
    num_class=set(Y)
    count_class={}
    for i in range(len(num_class)):
        count_class[i]=sum([1 for y in Y if y==i])
    return count_class

balance=balance_class(Y)
balance

(35887, 2304)
(35887,)
7


{0: 4953, 1: 547, 2: 5121, 3: 8989, 4: 6077, 5: 4002, 6: 6198}

In [10]:
#reshape X to fit keras with tensorflow backend
N,D = X.shape
X = X.reshape(N,48,48,1) # last dimension =1 is because it is black and white image, if colored, it will be 3

#split into training and testing set and rearrange the label y into 7 classes 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)
y_train= (np.arange(num_class) == y_train[:,None]).astype(np.float32)
y_test=(np.arange(num_class) == y_test[:,None]).astype(np.float32)

In [11]:
X_train, X_test, y_train, y_test

(array([[[[0.6627451 ],
          [0.67058824],
          [0.69019608],
          ...,
          [0.6627451 ],
          [0.65882353],
          [0.6627451 ]],
 
         [[0.67843137],
          [0.69803922],
          [0.69411765],
          ...,
          [0.67058824],
          [0.65490196],
          [0.6627451 ]],
 
         [[0.70196078],
          [0.70196078],
          [0.71372549],
          ...,
          [0.61176471],
          [0.6745098 ],
          [0.6627451 ]],
 
         ...,
 
         [[0.74117647],
          [0.71764706],
          [0.65882353],
          ...,
          [0.05098039],
          [0.27843137],
          [0.61568627]],
 
         [[0.75294118],
          [0.72941176],
          [0.65098039],
          ...,
          [0.02352941],
          [0.23529412],
          [0.63137255]],
 
         [[0.80392157],
          [0.74901961],
          [0.65098039],
          ...,
          [0.00784314],
          [0.21568627],
          [0.60784314]]],
 
 
        [

In [12]:
num_classes = num_class
num_classes

7

In [165]:
model = Sequential()
 
#1st convolution layer
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(5,5), strides=(2, 2)))
 
#2nd convolution layer
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(AveragePooling2D(pool_size=(3,3), strides=(2, 2)))
 
#3rd convolution layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(AveragePooling2D(pool_size=(3,3), strides=(2, 2)))
 
model.add(Flatten())
 
#fully connected neural networks
model.add(Dense(500, activation='relu')) # layer
model.add(Dropout(0.25))
model.add(Dense(500, activation='relu')) # layer
model.add(Dropout(0.25))
 
model.add(Dense(num_classes, activation='sigmoid')) # layer

In [166]:
rmsprop = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)

model.compile(loss='categorical_crossentropy',
              optimizer= rmsprop,
              metrics=['accuracy'])

In [176]:
##Train Model
nb_epoch = 8
nb_train_samples = 30
nb_validation_samples = 400

In [177]:
model.fit(X_train, y_train, 
            steps_per_epoch=nb_train_samples, 
            epochs=nb_epoch, 
            verbose=1, 
            validation_data=(X_test, y_test),
            validation_steps=nb_validation_samples)

print('Test loss:', score[0]) 
print('Test accuracy:', score[1]) 

Train on 28709 samples, validate on 7178 samples
Epoch 1/8
30/30 [==============================] - 6144s 205s/step - loss: 1.8194 - acc: 0.2491 - val_loss: 1.8073 - val_acc: 0.2559
Epoch 2/8
30/30 [==============================] - 5884s 196s/step - loss: 1.8048 - acc: 0.2491 - val_loss: 1.7805 - val_acc: 0.2559
Epoch 3/8
30/30 [==============================] - 6579s 219s/step - loss: 1.7888 - acc: 0.2491 - val_loss: 1.7584 - val_acc: 0.2559
Epoch 4/8
30/30 [==============================] - 5733s 191s/step - loss: 1.7593 - acc: 0.2597 - val_loss: 1.7254 - val_acc: 0.2784
Epoch 5/8
30/30 [==============================] - 5840s 195s/step - loss: 1.7258 - acc: 0.2894 - val_loss: 1.7230 - val_acc: 0.2789
Epoch 6/8
30/30 [==============================] - 5532s 184s/step - loss: 1.6972 - acc: 0.3104 - val_loss: 1.7513 - val_acc: 0.2919
Epoch 7/8
30/30 [==============================] - 6013s 200s/step - loss: 1.6642 - acc: 0.3324 - val_loss: 1.6031 - val_acc: 0.3543
Epoch 8/8
30/30 [===

NameError: name 'classifier' is not defined

In [178]:
model.evaluate(X_test, y_test, verbose=0) 

[1.5771978315984216, 0.3737809975006415]

In [179]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test Loss:', score[0])
print('Test accuracy:', score[1])

Test Loss: 1.5771978315984216
Test accuracy: 0.3737809975006415
